# RL - Exercício 2 - Taxi V3

## Trabalho desenvolvido durante o MBA em Data Science / IA na FIAP - 2022~2023.

## Prof. Felipe Teodoro

## Agente de Reinforcement Learning jogando Taxi V3 usando Q-Learning

Adaptado HuggingFace

<img src="https://www.gymlibrary.dev/_images/taxi.gif" alt="Environments"/>

### 🎮 Environments:

- [Taxi-v3](https://www.gymlibrary.dev/environments/toy_text/taxi/)


### 📚 RL-Library:

- Python and NumPy
- [Gym](https://www.gymlibrary.dev/)

## Instalar dependências e criar um display virtual 🔽


In [1]:
!pip install gym==0.24
!pip install pygame
!pip install numpy

!pip install huggingface_hub
!pip install pickle5
!pip install pyyaml==6.0
!pip install imageio
!pip install imageio_ffmpeg
!pip install pyglet==1.5.1
!pip install tqdm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/694.4 kB ? eta -:--:--
     ━━━━━━━━━╸━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 174.1/694.4 kB 5.3 MB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╺━━━ 624.6/694.4 kB 8.8 MB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 694.4/694.4 kB 7.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for gym: filename=gym-0.24.0-py3-none-any.whl size=790676 sha256=221ee56f633d44c770b17fbac0955c98fbc42f3fa9f798a6f0062c434918830c
  Stored in directory: /root/.cache/pip/wheels/2f/a1/b1/5f4110c8943b36e6cdfcc0e5768481639fa042b3c8357e82cc
Successfully built gym
  Attempting uninstall: gym
    Found existing installation: gym 0.25.2
    Uninstalling gym-0.25.2:
      Successfully uninstalled gym-0.25.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.

In [2]:
%%capture
!apt update
!apt install ffmpeg xvfb
!pip install xvfbwrapper
!pip install pyvirtualdisplay

Para garantir que as novas bibliotecas instaladas sejam usadas, **às vezes é necessário reiniciar o tempo de execução do notebook**. A próxima célula forçará o **tempo de execução a travar, então você precisará se conectar novamente e executar o código a partir daqui**.

In [3]:
#import os
#os.kill(os.getpid(), 9)

In [4]:
# Virtual display
from pyvirtualdisplay import Display

virtual_display = Display(visible=0, size=(1400, 900))
virtual_display.start()

## Importação de pacotes 📦

Além das bibliotecas instaladas, utilizamos também:

- `random`: Para gerar números aleatórios (que serão úteis para a política epsilon-greedy).
- `imageio`: Para gerar um vídeo de replay.

In [5]:
import numpy as np
import gym
import random
import imageio
import os
import tqdm
import time

import pickle5 as pickle
from tqdm.notebook import tqdm

In [6]:
env = gym.make("Taxi-v3")

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


### Verifique o Environment:


In [7]:
print("_____OBSERVATION SPACE_____ \n")
print("Observation Space", env.observation_space)
print("Sample observation", env.observation_space.sample()) # Get a random observation

_____OBSERVATION SPACE_____ 

Observation Space Discrete(500)
Sample observation 273


In [8]:
print("\n _____ACTION SPACE_____ \n")
print("Action Space Shape", env.action_space.n)
print("Action Space Sample", env.action_space.sample()) # Take a random action


 _____ACTION SPACE_____ 

Action Space Shape 6
Action Space Sample 4


O espaço de ação - conjunto de ações possíveis que o agente pode realizar - é discreto com 4 ações disponíveis 🎮:

- 0: vá para o sul
- 1: vá para o norte
- 2: vá para o leste
- 3: vá para o oeste
- 4: pegar passageiro
- 5: deixar passageiro

Função de recompensa 💰:

- Cada passo dados, a não ser que haja outra recompensa: -1
- Deixar o passageiro no lugar certo: +20
- Deixar ou pegar o passageiro em local errado: -10

### Sobre o espaço de observações:

O espaço de observações (estados) tem dimensão 500 porque o ambiente consiste em uma matriz 5x5, onde cada célula representa uma localização possível, porém, além disso, há 4 locais possíveis para o passageiro (4 possíveis posições de pick-up) e 5 locais possíveis para o destino (4 posições de pick-up mais uma posição no táxi quando o destino foi atingido). Portanto, o total de combinações de estados possíveis é 5 x 5 x 5 x 4 = 500.

In [9]:
state_space = env.observation_space.n
print("There are ", state_space, " possible states")

action_space = env.action_space.n
print("There are ", action_space, " possible actions")

There are  500  possible states
There are  6  possible actions


In [10]:
# Vamos criar nossa Qtable de tamanho (state_space, action_space) e inicializar cada valor em 0 usando np.zeros
def initialize_q_table(state_space, action_space):
  Qtable = np.zeros((state_space, action_space))
  return Qtable

In [11]:
Qtable_taxiv3 = initialize_q_table(state_space, action_space)

In [12]:
Qtable_taxiv3

array([[0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       ...,
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.]])

In [13]:
Qtable_taxiv3.shape

(500, 6)

In [14]:
def greedy_policy(Qtable, state):
  # Exploitation: take the action with the highest state, action value
  action = np.argmax(Qtable[state][:])

  return action

In [15]:
def epsilon_greedy_policy(Qtable, state, epsilon):
  # Gera aleatoriamente um número entre 0 e 1
  random_int = random.uniform(0,1)
  # if random_int > maior que epsilon --> exploitation
  if random_int > epsilon:
     # Execute a ação com o maior valor dado um estado
     # np.argmax pode ser útil aqui
    action = greedy_policy(Qtable, state)
  # else --> exploration
  else:
    action = env.action_space.sample()

  return action

## Definindo os hiperparâmetros ⚙️
---


Os hiperparâmetros relacionados à exploração são alguns dos mais importantes.

- Precisamos garantir que nosso agente **explore o espaço de estados** o suficiente para aprender uma boa aproximação de valor. Para fazer isso, precisamos ter decaimento progressivo do epsilon.
- Se você diminuir o epsilon muito rápido (decay_rate muito alto), **você corre o risco de que seu agente fique preso**, já que seu agente não explorou o espaço de estado o suficiente e, portanto, não pode resolver o problema.

Veja como cheguei nos parâmetros abaixo no [gridsearch que montei aqui](#scrollTo=am_OP5Me9bxZ&line=1&uniqifier=1) (vide mais abaixo neste notebook)

In [16]:
# Parâmetros de treinamento
n_training_episodes = 10000 # Total de episódios de treinamento
''' Anotação da aula
- um episódio de treinamento é uma "rodada" completa, uma
  tentativa de chegar no prêmio. Termina quando o bonequinho cai no buraco,
  no gelo ou quando chega no presente.
'''
learning_rate = 0.7 # Taxa de aprendizado
''' Anotação da aula
- taxa de aprendizado não é fácil de calibrar e não tem método. Por convenção
  coloca-se algo entre 0.7 e 0.9
'''

# Parâmetros de avaliação
n_eval_episodes = 100 # Número total de episódios de teste

# Parâmetros do ambiente
env_id = "Taxi-v3" # Nome do ambiente

max_steps = 101 # Max passos por episódio
''' Anotação da aula
- Evita o carrinho ficar em loop infinito. Se em X passos, não acabar,
  encerra o episódio
'''
gamma = 0.90 # Taxa de desconto
eval_seed = [73] * n_eval_episodes # A semente de avaliação do ambiente

# Parâmetros de exploração
max_epsilon = 1.0 # Probabilidade de exploração no início
min_epsilon = 0.05 # Probabilidade mínima de exploração
decay_rate = 0.0001 # Taxa de decaimento exponencial para prob de exploração

## Treinando o agente Q-Learning 🏃

In [17]:
def train(n_training_episodes, min_epsilon, max_epsilon, decay_rate, env, max_steps, Qtable):

  print("Training with ", n_training_episodes, min_epsilon, max_epsilon, decay_rate, max_steps, gamma, learning_rate)

  max_step_warning = False
  episodes_ = []

  for episode in tqdm(range(n_training_episodes)):
    # # Reduzir epsilon (porque precisamos cada vez menos exploration)
    epsilon = min_epsilon + (max_epsilon - min_epsilon)*np.exp(-decay_rate*episode)
    # Redefinir o ambiente
    state = env.reset()
    step = 0
    done = False

    # repete
    for step in range(max_steps):
      # Escolha a ação At para usar a política gananciosa (greedy policy) do epsilon
      action = epsilon_greedy_policy(Qtable, state, epsilon)

      # Take action At and observe Rt+1 and St+1
      # Take the action (a) and observe the outcome state(s') and reward (r)
      new_state, reward, done, info = env.step(action)

      # Update Q(s,a):= Q(s,a) + lr [R(s,a) + gamma * max Q(s',a') - Q(s,a)]
      Qtable[state][action] = Qtable[state][action] + learning_rate * (reward + gamma * np.max(Qtable[new_state]) - Qtable[state][action])

      # If done, finish the episode
      if done:
        break

      # Our next state is the new state
      state = new_state

    if (not max_step_warning and not done and step == (max_steps - 1)):
      max_step_warning = True
      episodes_.append(episode)

  if max_step_warning:
    print('Episodes reached max steps before done (', episodes_, ')')

  return Qtable

## Verificando a tabela Q-Learning 👀

In [18]:
Qtable_taxiv3 = train(n_training_episodes, min_epsilon, max_epsilon, decay_rate, env, max_steps, Qtable_taxiv3)

Training with  10000 0.05 1.0 0.0001 101 0.9 0.7


  0%|          | 0/10000 [00:00<?, ?it/s]

Episodes reached max steps before done ( [0] )


In [19]:
Qtable_taxiv3

array([[ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ],
       [-0.58568212,  0.4603532 , -0.58568212,  0.4603532 ,  1.62261467,
        -8.5396468 ],
       [ 4.348907  ,  5.94323   ,  4.348907  ,  5.94323   ,  7.7147    ,
        -3.05677   ],
       ...,
       [ 7.7147    ,  9.683     ,  7.7147    ,  5.94323   , -1.2853    ,
        -1.2853    ],
       [ 1.62261467,  2.9140163 ,  1.62261467,  2.9140163 , -7.37738533,
        -7.37738533],
       [14.3       , 11.87      , 14.3       , 17.        ,  5.3       ,
         5.3       ]])

## Avaliação do Método 📝

- Definimos o método de avaliação que vamos usar para testar nosso agente Q-Learning.

In [20]:
def evaluate_agent(env, max_steps, n_eval_episodes, Q, seed):
  """
   Avalie o agente para episódios ``n_eval_episodes`` e retorne recompensa média e padrão de recompensa.
   :param env: O ambiente de avaliação
   :param n_eval_episodes: Número de episódios para avaliar o agente
   :param Q: A tabela Q
   :param seed: A matriz de sementes de avaliação (para taxi-v3)
   """
  episode_rewards = []
  for episode in tqdm(range(n_eval_episodes)):
    if seed:
      state = env.reset(seed=seed[episode])
    else:
      state = env.reset()
    step = 0
    done = False
    total_rewards_ep = 0

    for step in range(max_steps):
      # Tome a ação (índice) que tem a recompensa futura máxima esperada dado aquele estado
      action = greedy_policy(Q, state)
      new_state, reward, done, info = env.step(action)
      total_rewards_ep += reward

      if done:
        break
      state = new_state
    episode_rewards.append(total_rewards_ep)
  mean_reward = np.mean(episode_rewards)
  std_reward = np.std(episode_rewards)

  return mean_reward, std_reward

## Avaliando nosso agente Q-Learning 📈


In [21]:
# Evaluate our Agent
mean_reward, std_reward = evaluate_agent(env, max_steps, n_eval_episodes, Qtable_taxiv3, eval_seed)
print(f"Mean_reward={mean_reward:.2f} +/- {std_reward:.2f}")

  0%|          | 0/100 [00:00<?, ?it/s]

Mean_reward=7.00 +/- 0.00


## Procurando Hiperparâmetros Ótimos para o Agente Q-Learning 💡


In [22]:

# "Gridsearch" para encontrar melhores hiperparâmetros

best_mean_reward = float('-inf')
best_std_reward = float('inf')
best_hyperparams = None

learning_rates = [0.7, 0.8, 0.9]
gammas = [0.9, 0.95, 0.99]
decay_rates = [0.0001, 0.0005, 0.005]
max_steps_ =  [101, 202, 404]

for learning_rate in learning_rates:
  for gamma in gammas:
    for decay_rate in decay_rates:
      for max_steps in max_steps_:
        Qtable_taxiv3 = train(n_training_episodes, min_epsilon, max_epsilon, decay_rate, env, max_steps, Qtable_taxiv3)
        mean_reward, std_reward = evaluate_agent(env, max_steps, n_eval_episodes, Qtable_taxiv3, eval_seed)

        # Verificar se essa configuração é a melhor até agora
        if mean_reward > best_mean_reward:
          best_mean_reward = mean_reward
          best_std_reward = std_reward
          best_hyperparams = (learning_rate, gamma, decay_rate, max_steps)

print("Melhor desempenho:")

print("Mean Reward:", best_mean_reward)
print("Std Reward:", best_std_reward)
print("Hyperparameters:", best_hyperparams)


Training with  10000 0.05 1.0 0.0001 101 0.9 0.7


  0%|          | 0/10000 [00:00<?, ?it/s]

Episodes reached max steps before done ( [0] )


  0%|          | 0/100 [00:00<?, ?it/s]

Training with  10000 0.05 1.0 0.0001 202 0.9 0.7


...

  0%|          | 0/100 [00:00<?, ?it/s]

Training with  10000 0.05 1.0 0.005 404 0.99 0.9


  0%|          | 0/10000 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

Melhor desempenho:
Mean Reward: 7.0
Std Reward: 0.0
Hyperparameters: (0.7, 0.9, 0.0001, 101)


Hiperparâmetros ótimos encontrados aqui:

```
learning_rate = 0.7
gamma = 0.90
decay_rate = 0.0001
max_steps = 101
```

Configuro os hiperparâmetros inciais com estes dados [aqui](#scrollTo=g1NLX6pr1j8Y&line=1&uniqifier=1) (vide mais acima neste notebook, na definição dos hiperparâmetros para o treinamento do agente)

#### Não modifique essa parte


In [26]:
def record_video(env, Qtable, out_directory, fps=1):
  """
   Gerar um vídeo de replay do agente
   :param env
   :param Qtable: Qtable do nosso agente
   :param out_directory
   :param fps: quantos quadros por segundo (com taxi-v3 e frozenlake-v1 usamos 1)
   """
  images = []
  done = False
  state = env.reset()
  img = env.render(mode='rgb_array')
  images.append(img)
  while not done:
    # Tome a ação (índice) que tem a recompensa futura máxima esperada dado aquele estado
    action = np.argmax(Qtable[state][:])
    state, reward, done, info = env.step(action) # Colocamos diretamente next_state = state para a lógica de gravação
    img = env.render(mode='rgb_array')
    images.append(img)
  imageio.mimsave(out_directory, [np.array(img) for i, img in enumerate(images)], fps=fps)


In [27]:
video_path =  "replay.mp4"
record_video(env, Qtable_taxiv3, video_path, 2)

In [28]:
from IPython.display import HTML
from base64 import b64encode
import os


# Show video
mp4 = open(video_path,'rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML("""
<video width=400 controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)